In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [13]:
df = pd.read_csv(r'C:\Users\arjun\Downloads\dataset-3.csv' )

In [14]:
df.shape

(44, 3)

In [7]:
df.head()

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1


Question 1: Distance Matrix Calculation
Create a function named calculate_distance_matrix that takes the dataset-3.csv as input and generates a DataFrame representing distances between IDs.

The resulting DataFrame should have cumulative distances along known routes, with diagonal values set to 0. If distances between toll locations A to B and B to C are known, then the distance from A to C should be the sum of these distances. Ensure the matrix is symmetric, accounting for bidirectional distances between toll locations (i.e. A to B is equal to B to A).

In [8]:

def calculate_distance_matrix(df):
    # Assuming the dataset-3.csv has columns 'id_start', 'id_end', and 'distance'
    distance_matrix = pd.pivot_table(df, values='distance', index='id_start', columns='id_end', fill_value=0)
    distance_matrix = distance_matrix + distance_matrix.transpose()
    distance_matrix.values[[range(distance_matrix.shape[0])]*2] = 0
    for col in distance_matrix.columns:
        for idx in distance_matrix.index:
            if pd.isnull(distance_matrix.loc[idx, col]):
                distance_matrix.loc[idx, col] = distance_matrix.loc[col, idx]
    
    return distance_matrix

In [12]:

distance_result = calculate_distance_matrix(df)
print(distance_result)

         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1001402      NaN      0.0     20.2      0.0      0.0      0.0      0.0   
1001404      NaN     20.2      0.0     16.0      0.0      0.0      0.0   
1001406      NaN      0.0     16.0      0.0     21.7      0.0      0.0   
1001408      NaN      0.0      0.0     21.7      0.0     11.1      0.0   
1001410      NaN      0.0      0.0      0.0     11.1      0.0     15.6   
1001412      NaN      0.0      0.0      0.0      0.0     15.6      0.0   
1001414      NaN      0.0      0.0      0.0      0.0      0.0     18.2   
1001416      NaN      0.0      0.0      0.0      0.0      0.0      0.0   
1001418      NaN      0.0      0.0      0.0      0.0      0.0      0.0   
1001420      NaN      0.0      0.0      0.0      0.0      0.0      0.0   
1001422      NaN      0.0      0.0      0.0      0.0      0.0      0.0   
1001424      NaN      0.0      0.0    

Question 2: Unroll Distance Matrix
Create a function unroll_distance_matrix that takes the DataFrame created in Question 1. The resulting DataFrame should have three columns: columns id_start, id_end, and distance.

All the combinations except for same id_start to id_end must be present in the rows with their distance values from the input DataFrame.

In [15]:
def unroll_distance_matrix(distance_matrix):
    unrolled_distance = distance_matrix.unstack().reset_index()
    unrolled_distance.columns = ['id_start', 'id_end', 'distance']
    unrolled_distance = unrolled_distance[unrolled_distance['id_start'] != unrolled_distance['id_end']]
    return unrolled_distance

In [18]:

distance_result = unroll_distance_matrix(df)
print(distance_result)

     id_start  id_end   distance
0    id_start       0  1001400.0
1    id_start       1  1001402.0
2    id_start       2  1001404.0
3    id_start       3  1001406.0
4    id_start       4  1001408.0
..        ...     ...        ...
127  distance      39       26.7
128  distance      40        8.5
129  distance      41       10.7
130  distance      42       10.6
131  distance      43       16.0

[132 rows x 3 columns]


Question 3: Finding IDs within Percentage Threshold
Create a function find_ids_within_ten_percentage_threshold that takes the DataFrame created in Question 2 and a reference value from the id_start column as an integer.

Calculate average distance for the reference value given as an input and return a sorted list of values from id_start column which lie within 10% (including ceiling and floor) of the reference value's average.

In [19]:
def find_ids_within_ten_percentage_threshold(distance_data, reference_id):
    reference_avg_distance = distance_data[distance_data['id_start'] == reference_id]['distance'].mean()
    threshold_min = reference_avg_distance * 0.9
    threshold_max = reference_avg_distance * 1.1
    within_threshold_ids = distance_data[(distance_data['id_start'] != reference_id) & 
                                         (distance_data['distance'] >= threshold_min) & 
                                         (distance_data['distance'] <= threshold_max)]['id_start'].unique()
    within_threshold_ids = list(within_threshold_ids)-
    within_threshold_ids.sort()
    return within_threshold_ids

Question 4: Calculate Toll Rate
Create a function calculate_toll_rate that takes the DataFrame created in Question 2 as input and calculates toll rates based on vehicle types.

The resulting DataFrame should add 5 columns to the input DataFrame: moto, car, rv, bus, and truck with their respective rate coefficients. The toll rates should be calculated by multiplying the distance with the given rate coefficients for each vehicle type:

0.8 for moto
1.2 for car
1.5 for rv
2.2 for bus
3.6 for truck

In [25]:
def calculate_toll_rate(vehicle_data):
    vehicle_data['moto'] = vehicle_data['distance'] * 0.8
    vehicle_data['car'] = vehicle_data['distance'] * 1.2
    vehicle_data['rv'] = vehicle_data['distance'] * 1.5
    vehicle_data['bus'] = vehicle_data['distance'] * 2.2
    vehicle_data['truck'] = vehicle_data['distance'] * 3.6
    return vehicle_data

In [27]:

calculate_toll = calculate_toll_rate(df)
print(calculate_toll)

    id_start   id_end  distance   moto    car     rv    bus  truck
0    1001400  1001402       9.7   7.76  11.64  14.55  21.34  34.92
1    1001402  1001404      20.2  16.16  24.24  30.30  44.44  72.72
2    1001404  1001406      16.0  12.80  19.20  24.00  35.20  57.60
3    1001406  1001408      21.7  17.36  26.04  32.55  47.74  78.12
4    1001408  1001410      11.1   8.88  13.32  16.65  24.42  39.96
5    1001410  1001412      15.6  12.48  18.72  23.40  34.32  56.16
6    1001412  1001414      18.2  14.56  21.84  27.30  40.04  65.52
7    1001414  1001416      13.2  10.56  15.84  19.80  29.04  47.52
8    1001416  1001418      13.6  10.88  16.32  20.40  29.92  48.96
9    1001418  1001420      12.9  10.32  15.48  19.35  28.38  46.44
10   1001420  1001422       9.6   7.68  11.52  14.40  21.12  34.56
11   1001422  1001424      11.4   9.12  13.68  17.10  25.08  41.04
12   1001424  1001426      18.6  14.88  22.32  27.90  40.92  66.96
13   1001426  1001428      15.8  12.64  18.96  23.70  34.76  5

Question 5: Calculate Time-Based Toll Rates
Create a function named calculate_time_based_toll_rates that takes the DataFrame created in Question 3 as input and calculates toll rates for different time intervals within a day.

The resulting DataFrame should have these five columns added to the input: start_day, start_time, end_day, and end_time.

start_day, end_day must be strings with day values (from Monday to Sunday in proper case)
start_time and end_time must be of type datetime.time() with the values from time range given below.
Modify the values of vehicle columns according to the following time ranges:

Weekdays (Monday - Friday):

From 00:00:00 to 10:00:00: Apply a discount factor of 0.8
From 10:00:00 to 18:00:00: Apply a discount factor of 1.2
From 18:00:00 to 23:59:59: Apply a discount factor of 0.8
Weekends (Saturday and Sunday):

Apply a constant discount factor of 0.7 for all times.
For each unique (id_start, id_end) pair, cover a full 24-hour period (from 12:00:00 AM to 11:59:59 PM) and span all 7 days of the week (from Monday to Sunday).

In [29]:


def calculate_time_based_toll_rates(vehicle_data):
    # Create a new DataFrame to store the time-based toll rates
    time_based_toll_rates = vehicle_data.copy()

    # Define the time ranges for weekdays and weekends
    weekday_time_ranges = [
        (datetime.time(0, 0, 0), datetime.time(10, 0, 0)),
        (datetime.time(10, 0, 0), datetime.time(18, 0, 0)),
        (datetime.time(18, 0, 0), datetime.time(23, 59, 59))
    ]
    weekend_time_range = (datetime.time(0, 0, 0), datetime.time(23, 59, 59))

    # Iterate through each row in the DataFrame
    for index, row in time_based_toll_rates.iterrows():
        # Get the day of the week for the start and end times
        start_day = row['start_time'].strftime('%A')
        end_day = row['end_time'].strftime('%A')

        # Apply the discount factor based on the time range and day of the week
        if start_day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
            for start_time, end_time in weekday_time_ranges:
                if start_time <= row['start_time'].time() <= end_time and start_time <= row['end_time'].time() <= end_time:
                    if start_time == datetime.time(0, 0, 0) and end_time == datetime.time(10, 0, 0):
                        time_based_toll_rates.loc[index, 'moto'] *= 0.8
                        time_based_toll_rates.loc[index, 'car'] *= 0.8
                        time_based_toll_rates.loc[index, 'rv'] *= 0.8
                        time_based_toll_rates.loc[index, 'bus'] *= 0.8
                        time_based_toll_rates.loc[index, 'truck'] *= 0.8
                    elif start_time == datetime.time(10, 0, 0) and end_time == datetime.time(18, 0, 0):
                        time_based_toll_rates.loc[index, 'moto'] *= 1.2
                        time_based_toll_rates.loc[index, 'car'] *= 1.2
                        time_based_toll_rates.loc[index, 'rv'] *= 1.2
                        time_based_toll_rates.loc[index, 'bus'] *= 1.2
                        time_based_toll_rates.loc[index, 'truck'] *= 1.2
                    elif start_time == datetime.time(18, 0, 0) and end_time == datetime.time(23, 59, 59):
                        time_based_toll_rates.loc[index, 'moto'] *= 0.8
                        time_based_toll_rates.loc[index, 'car'] *= 0.8
                        time_based_toll_rates.loc[index, 'rv'] *= 0.8
                        time_based_toll_rates.loc[index, 'bus'] *= 0.8
                        time_based_toll_rates.loc[index, 'truck'] *= 0.8
        else:  # Saturday and Sunday
            time_based_toll_rates.loc[index, 'moto'] *= 0.7
            time_based_toll_rates.loc[index, 'car'] *= 0.7
            time_based_toll_rates.loc[index, 'rv'] *= 0.7
            time_based_toll_rates.loc[index, 'bus'] *= 0.7
            time_based_toll_rates.loc[index, 'truck'] *= 0.7

        # Add start_day and end_day columns to the DataFrame
        time_based_toll_rates.loc[index, 'start_day'] = start_day
        time_based_toll_rates.loc[index, 'end_day'] = end_day

    # Convert start_time and end_time columns to datetime.time() type
    time_based_toll_rates['start_time'] = pd.to_datetime(time_based_toll_rates['start_time']).dt.time
    time_based_toll_rates['end_time'] = pd.to_datetime(time_based_toll_rates['end_time']).dt.time

    return time_based_toll_rates


In [32]:
    weekday_time_ranges = [
        (datetime.time(0, 0, 0), datetime.time(10, 0, 0)),
        (datetime.time(10, 0, 0), datetime.time(18, 0, 0)),
        (datetime.time(18, 0, 0), datetime.time(23, 59, 59))
    ]
    weekend_time_range = (datetime.time(0, 0, 0), datetime.time(23, 59, 59))
print(weekday_time_ranges)

[(datetime.time(0, 0), datetime.time(10, 0)), (datetime.time(10, 0), datetime.time(18, 0)), (datetime.time(18, 0), datetime.time(23, 59, 59))]


In [ ]:


def calculate_time_based_toll_rates(vehicle_data):
    # Create a new DataFrame to store the time-based toll rates
    time_based_toll_rates = vehicle_data.copy()

    # Define the time ranges for weekdays and weekends
    weekday_time_ranges = [
        (datetime.time(0, 0, 0), datetime.time(10, 0, 0)),
        (datetime.time(10, 0, 0), datetime.time(18, 0, 0)),
        (datetime.time(18, 0, 0), datetime.time(23, 59, 59))
    ]
    weekend_time_range = (datetime.time(0, 0, 0), datetime.time(23, 59, 59))

    # Iterate through each row in the DataFrame
    for index, row in time_based_toll_rates.iterrows():
        # Get the day of the week for the start and end times
        start_day = row['start_time'].strftime('%A')
        end_day = row['end_time'].strftime('%A')

        # Apply the discount factor based on the time range and day of the week
        if start_day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
            for start_time, end_time in weekday_time_ranges:
                if start_time <= row['start_time'].time() <= end_time and start_time <= row['end_time'].time() <= end_time:
                    if start_time == datetime.time(0, 0, 0) and end_time == datetime.time(10, 0, 0):
                        time_based_toll_rates.loc[index, 'moto'] *= 0.8
                        time_based_toll_rates.loc[index, 'car'] *= 0.8
                        time_based_toll_rates.loc[index, 'rv'] *= 0.8
                        time_based_toll_rates.loc[index, 'bus'] *= 0.8
                        time_based_toll_rates.loc[index, 'truck'] *= 0.8
                    elif start_time == datetime.time(10, 0, 0) and end_time == datetime.time(18, 0, 0):
                        time_based_toll_rates.loc[index, 'moto'] *= 1.2
                        time_based_toll_rates.loc[index, 'car'] *= 1.2
                        time_based_toll_rates.loc[index, 'rv'] *= 1.2
                        time_based_toll_rates.loc[index, 'bus'] *= 1.2
                        time_based_toll_rates.loc[index, 'truck'] *= 1.2
                    elif start_time == datetime.time(18, 0, 0) and end_time == datetime.time(23, 59, 59):
                        time_based_toll_rates.loc[index, 'moto'] *= 0.8
                        time_based_toll_rates.loc[index, 'car'] *= 0.8
                        time_based_toll_rates.loc[index, 'rv'] *= 0.8
                        time_based_toll_rates.loc[index, 'bus'] *= 0.8
                        time_based_toll_rates.loc[index, 'truck'] *= 0.8
        else:  # Saturday and Sunday
            time_based_toll_rates.loc[index, 'moto'] *= 0.7
            time_based_toll_rates.loc[index, 'car'] *= 0.7
            time_based_toll_rates.loc[index, 'rv'] *= 0.7
            time_based_toll_rates.loc[index, 'bus'] *= 0.7
            time_based_toll_rates.loc[index, 'truck'] *= 0.7

        # Add start_day and end_day columns to the DataFrame
        time_based_toll_rates.loc[index, 'start_day'] = start_day
        time_based_toll_rates.loc[index, 'end_day'] = end_day

    # Convert start_time and end_time columns to datetime.time() type
    time_based_toll_rates['start_time'] = pd.to_datetime(time_based_toll_rates['start_time']).dt.time
    time_based_toll_rates['end_time'] = pd.to_datetime(time_based_toll_rates['end_time']).dt.time

    return time_based_toll_rates
